##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Penguin データセットを使った単純な TFX パイプラインのチュートリアル

***単純な TFX パイプラインを実行する短編チュートリアル。***

注意: この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colab で実行」をクリックするだけです

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/penguin_simple.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

ノートブックを使ったこのチュートリアルでは、TFX パイプラインを作成し、単純な分類モデルに使用します。このパイプラインは、ExampleGen、Trainer、Pusher という 3 つの TFX 基本コンポーネントで構成されます。パイプラインには、データのインポート、モデルのトレーニング、トレーニングされたモデルのエクスポートという最低限の ML ワークフローが含まれます。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。

## セットアップ

まず、TFX Python パッケージをインストールし、モデルに使用するデータセットをダウンロードする必要があります。

### Pip のアップグレード

ローカルで実行する場合にシステムの Pip をアップグレードしないように、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップふれーどできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX をインストールする


In [ ]:
!pip install -U tfx

### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「ランタイムを再起動」ボタンをクリックするか、「ランタイム」&gt;「ランタイムの再起動...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

TensorFlow と TFX のバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 変数をセットアップする

パイプラインを定義するために使用する変数がいくつかあります。これらの変数は、必要に応じてカスタマイズすることが可能です。デフォルトでは、パイプラインからのすべての出力は、現在のディレクトリに生成されます。

In [ ]:
import os

PIPELINE_NAME = "penguin-simple"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### サンプルデータを準備する

チュートリアルの TFX パイプラインで使用するサンプルデータセットをダウンロードします。使用するデータセットは [Palmer Penguins データセット](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)で、他の [TFX の例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)でも使用されているデータセットです。

このデータセットには、4 つの数値特徴量があります。

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

すべての特徴量はすでに範囲 [0,1] を持つように正規化されています。ペンギンの `species`（種）を予測する分類モデルを構築します。

TFX ExampleGen はディレクトリから入力を読み取るため、ディレクトリを作成してデータセットをそれにコピーする必要があります。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

CSV ファイルを見てみましょう。

In [ ]:
!head {_data_filepath}

5 つの値が表示されます。`species` は 0、1、2 のいずれかで、すべての特徴量には 0～1 の範囲の値があります。

## パイプラインを作成する

TFX パイプラインは Python API を使用して定義されています。ここでは、以下の 3 つのコンポーネントで構成されるパイプラインを定義します。

- CsvExampleGen: データファイルを読み取り、それらをさらに処理するために TFX 内部フォーマットに変換します。様々なフォーマットに対応する [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) が複数あります。このチュートリアルでは、CSV ファイルを入力として取る CsvExampleGen を使用します。
- Trainer: ML モデルをトレーニングします。[Trainer コンポーネント](https://www.tensorflow.org/tfx/guide/trainer)には、ユーザーが指定するモデル定義コードが必要です。モデルのトレーニング方法と *saved_model* 形式での保存方法は、TensorFlow API で指定できます。
- Pusher: トレーニングしたモデルを TFX パイプラインの外部にコピーします。[Pusher コンポーネント](https://www.tensorflow.org/tfx/guide/pusher)は、トレーニングした ML モデルのデプロイプロセスとして捉えることができます。

パイプラインを実際に定義する前にまず、Trainer コンポーネントのモデルコードを書く必要があります。

### モデルトレーニングのコードを記述する

TensorFlow Keras API を使って、分類用の単純な DNN モデルを作成します。このモデルトレーニングのコードは別のファイルに保存されます。

このチュートリアルでは、Keras ベースのモデルをサポートする TFX の[汎用 Trainer](https://www.tensorflow.org/tfx/guide/trainer#generic_trainer)を使います。`run_fn` 関数を含む Python ファイルを書く必要があります。これが `Trainer` コンポーネントのエントリポイントです。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

これで、TFX パイプラインを構築するための準備手順がすべて完了しました。

### パイプライン定義を記述する

TFX パイプラインを作成する関数を定義します。`Pipeline` オブジェクトは、TFX がサポートするパイプラインオーケストレーションシステムを使って実行できる TFX パイプラインです。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## パイプラインの実行

TFX は、パイプラインを実行するオーケストレータを複数サポートしています。このチュートリアルでは、TFX Python パッケージに含まれており、ローカル環境でパイプラインを実行する `LocalDagRunner` を使用します。通常、TFX パイプラインは、有向非巡回グラフを意味する「DAG」と呼ばれています。

`LocalDagRunner` では、開発やデバッグでのイテレーションを素早く行います。TFX は、本番ユースケースに適した Kubeflow Pipelines や Apache Airflow など、他のオーケストレータもサポートしています。

他のオーケストレーションシステムについての詳細は、[Cloud AI Platform Pipelines での TFX](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines) や [TFX Airflow チュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop)をご覧ください。

では、`LocalDagRunner` を作成して、定義済みの関数から作成した `Pipeline` オブジェクトを渡しましょう。

このパイプラインは直接実行するため、ML モデルトレーニングを含む、パイプラインの進行状況のログを確認することができます。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

パイプラインが正常に完了したら、ログの最後に「INFO:absl:Component Pusher is finished.」と表示されます。`Pusher` コンポーネントがパイプラインの最後のコンポーネントであるためです。

Pusher コンポーネントは、トレーニングしたモデルを `SERVING_MODEL_DIR` にプッシュします。これは、前の手順で変数を変更していない場合は `serving_model/penguin-simple` ディレクトリとなります。Colab の左側のパセルにあるファイルブラウザーで、または以下のコマンドを使うと、結果を確認できます。

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

## 次のステップ

その他のリソースは、https://www.tensorflow.org/tfx/tutorials に掲載されています。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。
